In [ ]:
char_sentences = dict()
for character in df.character.unique():
    text = tokenize(df.sentence[df.character == character].str.cat(sep=' '))
    char_sentences[character] = text

# Get average sentiment for all sentences of character
char_sentiment = {}
for char in char_sentences:
    sentiment_for_char = sentiment(nltk.FreqDist(char_sentences[char]))
    if sentiment_for_char:
        char_sentiment[char] = sentiment_for_char

In [ ]:
# Add one between characters that are in the same scene

scene_movie_interactions = dict()

scene_interactions = dict()
for movie in df.movie_nr.unique():
    scene_interactions = dict()
    df_store = df[df.movie_nr == movie].groupby(['scene_id'])['character'].unique()
    for scene in df_store:
        for idx in range(len(scene)-1):
            for idx2 in range(idx+1,len(scene)):
                if (scene[idx],scene[idx2]) in scene_interactions:
                    scene_interactions[(scene[idx],scene[idx2])] += 1
                else:
                    if (scene[idx2],scene[idx]) in scene_interactions:
                        scene_interactions[(scene[idx2],scene[idx])] += 1
                    else:
                        scene_interactions[(scene[idx],scene[idx2])] = 1
    scene_movie_interactions[movie] = scene_interactions

In [ ]:
for x in scene_movie_interactions:
    # Create an undirected weighted graph
    G = nx.Graph()
    for edge in scene_interactions:
        edge_1 = edge[0]
        edge_2 = edge[1]
        weight = scene_interactions[edge]
        G.add_edges_from([(edge_1, edge_2, {'weight': weight})])
    G_GCC = G.subgraph(max(nx.connected_components(G), key=len))
    with open(f'./data/networks_pickled/movie_{x}.pkl', 'wb') as f:
        pickle.dump(G_GCC, f)
    